
Lecture du Dataset

In [ ]:
fichier = open("tales_dataset.txt", "r")
f=fichier.read()


Tri


In [ ]:
txt="Bonjour mesdames, messieurs !\n je m'appelle Hugues"

def tri(txt):
  l=""
  for j in txt:
    #for j in i:
    if (j.isalpha() or j.isdigit() or j.isspace() or j=="'" or (j in ['.',',',';','!','?',':'])):
      l+=j.lower()

  #On enlève les sauts de ligne
  b=l.replace('\n',' ').split(' ')
  c=[]
  for i in b:
    if i != "":
      c.append(i)

  #On sépare la ponctuation
  cponc=[]
  for i in c:
    b=False
    for k in ['.',',',';','!','?',':']:
      if k in i:
        if i[:-1]!='':
          cponc.append(i[:-1])
        cponc.append(k)
        b=True
        
    if b==False:
      cponc.append(i)

  
  return cponc


Préparation dataset

In [ ]:
L=tri(f)
memo=50
M=[]
N=[]
for i in range(len(L)-memo):
  M.append(L[i:i+memo])
  N.append(L[i+memo])
Mr=M
Nr=N

Tokenizer

In [ ]:
from keras.preprocessing.text import Tokenizer

t=Tokenizer(filters='')

t.fit_on_texts(L)

len_dict=len(t.word_index)
print(len_dict)

47064


In [ ]:
lenMr=len(Mr)
c=round(0.8*lenMr)

Mrs = t.texts_to_sequences(Mr)
Nrs = t.texts_to_sequences(Nr)

data_trains=Mrs[:c]
data_valids=Mrs[c:]
label_train=Nrs[:c]
label_valid=Nrs[c:]
#import tensorflow as tf

#label_train = tf.keras.utils.to_categorical(Nrs[:c],len_dict)
#label_valid = tf.keras.utils.to_categorical(Nrs[c:],len_dict)
print(label_train[0])
print(Mrs[111668])

[420]
[6, 26126, 4, 34, 11, 26127, 1, 246, 1, 21, 58, 2203, 9, 575, 2, 106, 9030, 4, 30, 426, 14, 10, 1931, 377, 43, 13, 157, 226, 117, 33, 346, 5, 1045, 4612, 3, 13, 42, 117, 685, 53, 21, 5, 33, 26, 3, 221, 4, 42, 2, 106]


Embedding

In [ ]:
from tensorflow import keras
from keras import models,layers
import matplotlib.pyplot as plt
import tensorflow as tf

i = layers.Input(shape=(memo,))
model = models.Sequential()
model.add(i)
model.add(layers.Embedding(len_dict+1,15))
model.add(layers.LSTM(10,return_sequences=False))
model.add(layers.Dense(32,activation="relu"))
model.add(layers.Dense(32,activation="relu"))
model.add(layers.Dense(len_dict,activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 15)            705975    
                                                                 
 lstm (LSTM)                 (None, 10)                1040      
                                                                 
 dense (Dense)               (None, 32)                352       
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 47064)             1553112   
                                                                 
Total params: 2,261,535
Trainable params: 2,261,535
Non-trainable params: 0
_________________________________________________________________


Compile

In [ ]:
model.compile(optimizer='adam',loss=keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
model.fit(x=data_trains,y=label_train,batch_size=8,epochs=5,validation_data=(data_valids,label_valid))

Epoch 1/5
268162/268162 [==============================] - 2582s 10ms/step - loss: 5.7200 - accuracy: 0.1805 - val_loss: nan - val_accuracy: 0.1536
Epoch 2/5
268162/268162 [==============================] - 2541s 9ms/step - loss: 5.6282 - accuracy: 0.1808 - val_loss: nan - val_accuracy: 0.1512
Epoch 3/5
202785/268162 [=====================>........] - ETA: 8:58 - loss: 5.5141 - accuracy: 0.1811

In [ ]:

def generationv1(phrase):
  tri_p=tri(phrase)
  seq_p=t.texts_to_sequences(tri_p)
  memo=50
  Mp=[]
  for i in range(len(seq_p)-memo):
    Mp.append(seq_p[i:i+memo])
  print(len(tri_p))
  res = model.predict(Mp)
  print(len(res))
  i=len(res)-1
  a = max(res[i])
  ind = list(res[i]).index(a)
  ttt = t.word_index
  b=list(ttt.keys())[ind]
  return b

def generationv2(phrase):
  phr=phrase
  for i in range(10):
    phr=phr+' '+generationv1(phr)
  return phr
generationv2("Lovely Ilonka There was once a king's son who told his father that he wished to marry. 'No, no!' said the king; 'you must not be in such a hurry. Wait till you have done some great deed. My father did not let me marry till I had won the golden sword you see me wear.' The prince was much disappointed, but he never dreamed of disobeying his father, and he began to think with all his might what he could do. Yes, indeed, I've lived long and been much about in the world, but I have never seen or heard anything of what you ask. Still, if you will wait till to-morrow I may be able to tell you something. Well, he waited")
#generation("Yes, indeed, I've lived long and been much about in the world, but I have never seen or heard anything of what you ask. Still, if you will wait till to-morrow I may be able to tell you something.")

141
3/3 [==============================] - 1s 7ms/step
91
142
3/3 [==============================] - 0s 7ms/step
92
143
3/3 [==============================] - 0s 5ms/step
93
144
3/3 [==============================] - 0s 12ms/step
94
145
3/3 [==============================] - 0s 5ms/step
95
146
3/3 [==============================] - 0s 7ms/step
96
147
4/4 [==============================] - 0s 23ms/step
97
148
4/4 [==============================] - 0s 5ms/step
98
149
4/4 [==============================] - 0s 8ms/step
99
150
4/4 [==============================] - 0s 9ms/step
100


"Lovely Ilonka There was once a king's son who told his father that he wished to marry. 'No, no!' said the king; 'you must not be in such a hurry. Wait till you have done some great deed. My father did not let me marry till I had won the golden sword you see me wear.' The prince was much disappointed, but he never dreamed of disobeying his father, and he began to think with all his might what he could do. Yes, indeed, I've lived long and been much about in the world, but I have never seen or heard anything of what you ask. Still, if you will wait till to-morrow I may be able to tell you something. Well, he waited on of and and moment and the and are and"